In [1]:
#dependencies and setup
import pandas as pd
import os
from openpyxl import Workbook
import numpy as np
import requests
import json
import re
from pprint import pprint
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
# SQLite dependencies
import sqlite3
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float

# Extraction

## School performance score 2020

In [ ]:
# create a browser instance using splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
time.sleep(1)

In [ ]:
# visit the NJ schools url
nj_school_rate_URL = 'https://patch.com/new-jersey/pointpleasant/every-nj-school-graded-2020-state-report-where-do-you-rank?fbclid=IwAR3F6wscWhMGHzRk2hy4MeZJ426hdUOFCF8qDjHzFiI7B3RfxshE9imZdyg'
browser.visit(nj_school_rate_URL)
time.sleep(1)

# create HTML object
html = browser.html

# parse HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# access html elements to get the link for the table data
ol = soup.find('ol')
li = ol.find_all('li')
# iframe = div.find('iframe')
# table_data_link = iframe['src'].split("?")[0]
pprint(li)
# print(table_data_link)

In [ ]:
records = []
for elem in li:
    records.append(elem.text.split(",",4))
print(records)

In [ ]:
# close the browser session    
browser.quit()

In [ ]:
temp = re.findall(r'\d+', records[i][2]) 
res = list(map(int, temp)) 
re.sub(r'[^A-Za-z0-9 ]+', '', s)

In [ ]:
county_name = []
district_name = []
school_name = []
rating = []
for i in range(len(records)):
    l =len(records[i])
    county_name.append(re.sub(r'[^A-Za-z ]+', '', records[i][l-1]).upper().strip())
    rating.append(re.sub(r'[^0-9.]','', records[i][l-1]))
    district_name.append(records[i][l-2].strip().upper())
    school_name.append(records[i][l-3].strip().upper())

In [ ]:
school_ratings_2020_df = pd.DataFrame({"county_name": county_name,
                                    "district_name": district_name,
                                    "school_name": school_name,
                                    "rating": rating})
school_ratings_2020_df.sort_values(by =["county_name", "district_name", "school_name"], ascending=True, inplace=True, ignore_index=True)
school_ratings_2020_df.drop(school_ratings_2020_df[school_ratings_2020_df.county_name =='CHARTERS'].index, inplace=True)
school_ratings_2020_df.head()

In [ ]:
len(school_ratings_2020_df)

## School performance score 2017-2018

In [ ]:
# create a browser instance using splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
time.sleep(1)

In [ ]:
# visit the Mars News url
nj_school_rate_URL = 'https://www.nj.com/education/2019/03/nj-just-graded-its-public-schools-from-0-100-heres-how-every-school-scored.html'
browser.visit(nj_school_rate_URL)
time.sleep(1)

# create HTML object
html = browser.html

# parse HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# access html elements to get the link for the table data
div = soup.find('div', class_ = 'rawhtml')
iframe = div.find('iframe')
table_data_link = iframe['src'].split("?")[0]
# pprint(iframe)
print(table_data_link)

In [ ]:
# go to the file page in S3
browser.visit(table_data_link)
time.sleep(1)

# create HTML object
html = browser.html

# parse HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
div = soup.find('div', class_ = 'download')
school_rating_file_url = div.find('a', href = True)['href']
print(school_rating_file_url)

In [ ]:
# close the browser session    
browser.quit()

In [ ]:
# import the csv from the URL into dataframe 
dtype_dic= {'COUNTY_CODE': str, 
            'DISTRICT_CODE' : str,
           "SCHOOL_CODE": str}
school_rating_df = pd.read_csv(school_rating_file_url, dtype = dtype_dic)
# school_data.columns
school_rating_df = school_rating_df[['COUNTY_CODE', 'COUNTY_NAME', 'DISTRICT_CODE', 'DISTRICT_NAME',
       'SCHOOL_CODE', 'SCHOOL_NAME', 'GRADESPAN', 'PRINCIPAL_NAME', 'ADDRESS',
       'CITY_STATE_ZIP', 'PHONE', 'EMAIL', 'WEBSITE', 'FACEBOOK', 'TWITTER',
       'CutOffScoreTargetedSupImp', 'Targeted_LowPerf', 'ELAProf_MetTarget',
       'MathProf_MetTarget', 'Grad4YR_MetTarget', 'Grad5YR_MetTarget',
       'ELAGrowth_MetStandard', 'MathGrowth_MetStandard', 'ELP_MetTarget',
       'CA_MetAverage', 'Targeted_ConsUnderPerf', 'SummativeRating', 'SummativeScore']]
school_rating_df.head()

In [ ]:
# drop charter school records
school_rating_df.drop(school_rating_df[school_rating_df.COUNTY_NAME =='CHARTERS'].index, inplace=True)

In [ ]:
# unique counties
school_rating_df['COUNTY_CODE'].nunique()

In [ ]:
# make all the column name lower case
school_rating_df.columns= school_rating_df.columns.str.lower()
school_rating_df.head()

### Offline copy import

In [ ]:
# File to Load (Remember to Change These)
# source = https://www.nj.com/education/2019/03/nj-just-graded-its-public-schools-from-0-100-heres-how-every-school-scored.html
# file_to_load = "data/school_scores_2017_18.csv"
# dtype_dic= {'COUNTY_CODE': str, 
#             'DISTRICT_CODE' : str,
#            "SCHOOL_CODE": str}
# Read school score File and store into Pandas data frame
# school_data = pd.read_csv(file_to_load, dtype = dtype_dic)
# school_data = school_data[['COUNTY_CODE', 'COUNTY_NAME', 'DISTRICT_CODE', 'DISTRICT_NAME',
#        'SCHOOL_CODE', 'SCHOOL_NAME', 'GRADESPAN', 'PRINCIPAL_NAME', 'ADDRESS',
#        'CITY_STATE_ZIP', 'PHONE', 'EMAIL', 'WEBSITE', 'FACEBOOK', 'TWITTER',
#        'CutOffScoreTargetedSupImp', 'Targeted_LowPerf', 'ELAProf_MetTarget',
#        'MathProf_MetTarget', 'Grad4YR_MetTarget', 'Grad5YR_MetTarget',
#        'ELAGrowth_MetStandard', 'MathGrowth_MetStandard', 'ELP_MetTarget',
#        'CA_MetAverage', 'Targeted_ConsUnderPerf', 'SummativeRating', 'SummativeScore']]
# school_data.head()

## Poverty 2019

In [ ]:
# URL for county poverty data
poverty_url = 'https://api.census.gov/data/timeseries/poverty/saipe?get=NAME,SAEMHI_PT,SAEMHI_MOE,SAEPOVALL_PT,SAEPOVALL_MOE,SAEPOVRTALL_MOE,SAEPOVRTALL_PT,STABREV&for=county:*&in=state:34&YEAR=2019'

In [ ]:
# API call to get countries metadata
poverty_data = requests.get(poverty_url).json()
pprint(poverty_data)

In [ ]:
# initialize empty lists
county_name = []
median_hh_income = []
median_hh_inc_moe = []
poverty_count = []
poverty_count_moe = []
poverty_rate = []
poverty_rate_moe = []
st_abb =[]
state_fips = []
county_fips = []

In [ ]:
# iterate through the response to populate the lists and create a dataframe
for i in range(1,len(poverty_data)):
    county_name.append(poverty_data[i][0])
    median_hh_income.append(poverty_data[i][1])
    median_hh_inc_moe.append(poverty_data[i][2])
    poverty_count.append(poverty_data[i][3])
    poverty_count_moe.append(poverty_data[i][4])
    poverty_rate.append(poverty_data[i][5])
    poverty_rate_moe.append(poverty_data[i][6])
    st_abb.append(poverty_data[i][7])
    state_fips.append(poverty_data[i][9])
    county_fips.append(poverty_data[i][10])
counties_poverty_df = pd.DataFrame({"county_name": county_name,
                                    "median_hh_income": median_hh_income,
                                    "median_hh_inc_moe": median_hh_inc_moe,
                                    "poverty_count": poverty_count,
                                    "poverty_count_moe": poverty_count_moe,
                                    "poverty_rate": poverty_rate,
                                    "poverty_rate_moe": poverty_rate_moe,
                                    "st_abb": st_abb,
                                    "state_fips": state_fips,
                                    "county_fips": county_fips})
counties_poverty_df.head()

In [ ]:
# Manipulating the column name to match across all the datasets
counties_poverty_df['county_name']= counties_poverty_df.county_name.str.replace("County","")
counties_poverty_df['county_name']= counties_poverty_df['county_name'].str.upper()
counties_poverty_df['county_name']= counties_poverty_df['county_name'].str.strip()
counties_poverty_df

In [ ]:
# unique county count
counties_poverty_df['county_name'].nunique()

## County crime reports 2020

In [ ]:
# create a browser instance using splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
time.sleep(1)

In [ ]:
# visit the Mars News url
crime_NJ_URL = 'https://www.njsp.org/ucr/current-crime-data.shtml'
browser.visit(crime_NJ_URL)
time.sleep(2)

# create HTML object
html = browser.html

# parse HTML with BeautifulSoup
soup = BeautifulSoup(html, 'lxml')

In [ ]:
# use splinter to click on the 'Continue' button to retrieve a full-size jpg url
browser.find_by_tag("input").last.click()
time.sleep(1)

In [ ]:
# create HTML object
html = browser.html
# parse HTML with BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
# fetch the table
table = soup.find('table')
# fetch the partial link for donwloadable file
download_item = table.find_all('a', href = True)
download_link= download_item[0]['href']
# create full download link
base_url= 'https://www.njsp.org/ucr/'
full_file_link = base_url+download_link
print(full_file_link)

In [ ]:
# close the browser session    
browser.quit()

In [ ]:
# create xlsx object with the file from the link
xls = pd.ExcelFile(full_file_link,
     engine='openpyxl')
# get all the worksheet names
crime_sheet = xls.sheet_names
print(crime_sheet)

In [ ]:
# unique county
len(crime_sheet[1:])

In [ ]:
# create separate dataframes for each worksheet
for i in range(1,len(crime_sheet)):
    globals()['crime_%s' % crime_sheet[i].lower().replace(" ", "_")] = pd.read_excel(full_file_link,
     engine='openpyxl',sheet_name=crime_sheet[i], skiprows=range(3)).dropna(how="all")
    globals()['crime_%s' % crime_sheet[i].lower().replace(" ", "_")]["county_name"] = '%s' % crime_sheet[i].upper()
    globals()['crime_%s' % crime_sheet[i].lower().replace(" ", "_")].drop(globals()['crime_%s' % crime_sheet[i].lower().replace(" ", "_")].tail(7).index, inplace = True)
    print('crime_%s' % crime_sheet[i].lower().replace(" ", "_"))

### Offline copy import

In [ ]:
# del [crime_atlantic,
# crime_bergen,
# crime_burlington,
# crime_camden,
# crime_cape_may,
# crime_cumberland,
# crime_essex,
# crime_gloucester,
# crime_hudson,
# crime_hunterdon,
# crime_mercer,
# crime_middlesex,
# crime_monmouth,
# crime_morris,
# crime_ocean,
# crime_passaic,
# crime_salem,
# crime_somerset,
# crime_sussex,
# crime_union,
# crime_warren]

In [ ]:
# source : https://www.njsp.org/ucr/current-crime-data1.shtml?agree=0
# xls = pd.ExcelFile(os.path.join("data", "2020_UCR_Jan-Sept_v2.xlsx"),
#      engine='openpyxl')

In [ ]:
# crime_sheet = xls.sheet_names
# crime_sheet

In [ ]:
# for i in range(1,len(crime_sheet)):
#     globals()['crime_%s' % crime_sheet[i].lower()] = pd.read_excel(os.path.join("data", "2020_UCR_Jan-Sept_v2.xlsx"),
#      engine='openpyxl',sheet_name=crime_sheet[i], skiprows=range(3)).dropna(how="all")
#     print('crime_%s' % crime_sheet[i].lower())

#### Atlantic crime

In [ ]:
crime_atlantic.head(-1)

In [ ]:
len(crime_atlantic)

In [ ]:
# concat all the dataframes
all_crime_df = pd.concat([crime_atlantic,
crime_bergen,
crime_burlington,
crime_camden,
crime_cape_may,
crime_cumberland,
crime_essex,
crime_gloucester,
crime_hudson,
crime_hunterdon,
crime_mercer,
crime_middlesex,
crime_monmouth,
crime_morris,
crime_ocean,
crime_passaic,
crime_salem,
crime_somerset,
crime_sussex,
crime_union,
crime_warren], ignore_index = True)

In [ ]:
# make all the column name lower case
all_crime_df.columns= all_crime_df.columns.str.lower()
all_crime_df.columns= all_crime_df.columns.str.replace(" ","_")
all_crime_df.head(10)

### Population dataset

In [ ]:
population_df = all_crime_df[["county_name","agency", "orinumber", "population"]].dropna(how="any")
population_df["population"] = population_df["population"].astype(int)
population_df.drop_duplicates(keep='first', inplace= True, ignore_index=True)
population_df.head(-10)

In [ ]:
# unique county count
population_df['county_name'].nunique()

In [ ]:
len(population_df)

### Crime detail

In [ ]:
pol_dep = []
for i in range(len(population_df["agency"])):
    for k in range(5):
        pol_dep.append(population_df["agency"][i])
print(pol_dep)

In [ ]:
len(pol_dep)

In [ ]:
crime_det_df = all_crime_df[["county_name","agency", "murder", "rape", "robbery", "assault", "burglary",\
                         "larceny", "auto_theft", "total"]].dropna(how="any")

crime_det_df.rename(columns = {"agency": "report_type"}, inplace =True)

crime_det_df = crime_det_df.loc[crime_det_df["report_type"] != crime_det_df["report_type"].shift()]
crime_det_df.reset_index(inplace= True, drop = True)
crime_det_df["police_dept"] = pol_dep
crime_det_df = crime_det_df[["county_name","police_dept", "report_type", "murder", "rape", "robbery", "assault", "burglary",\
                         "larceny", "auto_theft", "total"]]
crime_det_df.head(-1)

### Crime summary dataset

In [ ]:
crime_df = all_crime_df[["county_name","agency", "murder", "rape", "robbery", "assault", "burglary",\
                         "larceny", "auto_theft", "total"]].dropna(how="any")
crime_df.reset_index(inplace= True, drop = True)
crime_df.rename(columns = {"agency": "report_type"}, inplace =True)
crime_df = crime_df.groupby(['county_name', 'report_type'],as_index=False).agg(murder = ('murder', 'sum'), rape = ('rape', 'sum'),\
                                                               robbery = ('robbery', 'sum'),assault = ('assault', 'sum'),\
                                                               burglary = ('burglary', 'sum'),larceny = ('larceny', 'sum'),\
                                                               auto_theft = ('auto_theft', 'sum'),total = ('total', 'sum'))
crime_df.head(-10)

In [ ]:
# unique county count
crime_df['county_name'].nunique()

In [ ]:
# county record count
crime_df['county_name'].value_counts()

# DB load

In [ ]:
# Connect to the NJ_CPS database
connection  = sqlite3.connect("Resources/NJ_CPS.sqlite")

# Get a cursor object
cursor = connection.cursor()

# Execute the DROP Table SQL statement
drop_school = "DROP TABLE IF EXISTS NJ_school_rating"

cursor.execute(drop_school)

drop_poverty = "DROP TABLE IF EXISTS NJ_poverty"

cursor.execute(drop_poverty)

drop_population = "DROP TABLE IF EXISTS NJ_population"

cursor.execute(drop_population)

drop_crime = "DROP TABLE IF EXISTS NJ_crime"

cursor.execute(drop_crime)

drop_crime_det = "DROP TABLE IF EXISTS NJ_crime_detail"

cursor.execute(drop_crime_det)

In [3]:
#  SQLite DB creation and establishing connection
database_path = "Resources/NJ_CPS.sqlite"
engine = create_engine(f"sqlite:///{database_path}", echo=True)
sqlite_connection = engine.connect()

2021-03-11 02:21:02,635 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-03-11 02:21:02,638 INFO sqlalchemy.engine.base.Engine ()
2021-03-11 02:21:02,640 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-03-11 02:21:02,641 INFO sqlalchemy.engine.base.Engine ()


## Loading school ratings in to DB

In [ ]:
sqlite_table = "NJ_school_rating"
school_rating_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

## Loading poverty rates in DB

In [ ]:
sqlite_table = "NJ_poverty"
counties_poverty_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

## Loading crime detail data in DB

In [ ]:
sqlite_table = "NJ_crime_detail"
crime_det_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

## Loading crime data in DB

In [ ]:
sqlite_table = "NJ_crime"
crime_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

## Loading population data in DB

In [ ]:
sqlite_table = "NJ_population"
population_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

# DB query Test

In [ ]:
metadata_df = pd.read_sql_query("SELECT T1.*, T2.population FROM (SELECT county_name,median_hh_income,poverty_rate FROM NJ_poverty) AS T1 \
                                INNER JOIN (SELECT county_name, sum(population) as population FROM NJ_population GROUP BY county_name) AS T2\
                                ON T1.county_name = T2.county_name", sqlite_connection)
metadata_df["median_hh_income"] = metadata_df["median_hh_income"].astype(int)
metadata_df

In [ ]:
school_df = pd.read_sql_query("SELECT county_name,district_code||school_code as school_id, gradespan, school_name,summativescore FROM NJ_school_rating", sqlite_connection)
school_df.head()

In [ ]:
crime_df = pd.read_sql_query("SELECT county_name, murder,rape, robbery, assault, burglary, larceny, auto_theft, total FROM NJ_crime WHERE report_type = 'Number of Offenses'", sqlite_connection)
crime_df.head()

In [ ]:
query = '''SELECT POV.county_name, POV.median_hh_income, POV.poverty_rate,
           CRM.murder, CRM.rape, CRM.robbery, CRM.assault, CRM.burglary, CRM.larceny, CRM.auto_theft, CRM.total,
           POP.population, SCH.school_rank
           FROM NJ_poverty AS POV 
           INNER JOIN NJ_crime AS CRM  ON POV.county_name = CRM.county_name AND report_type = 'Number of Offenses'
           INNER JOIN (SELECT county_name, SUM(population) AS population FROM NJ_population GROUP BY county_name) AS POP ON POV.county_name = POP.county_name
           INNER JOIN (SELECT county_name, AVG(summativescore) AS school_rank FROM NJ_school_rating GROUP BY county_name) AS SCH ON POV.county_name = SCH.county_name'''
test = pd.read_sql_query(query, sqlite_connection)
test

In [ ]:
query = '''SELECT POV.county_name, POV.median_hh_income, POV.poverty_rate, POV.county_fips,
               CRM.total AS total_offense, CRM2.total AS rate_per_100k, CRM3.total AS total_arrest,
               POP.population, SCH.school_rank
               FROM NJ_poverty AS POV 
               INNER JOIN NJ_crime AS CRM ON POV.county_name = CRM.county_name AND CRM.report_type = 'Number of Offenses'
               INNER JOIN NJ_crime AS CRM2 ON POV.county_name = CRM2.county_name AND CRM2.report_type = 'Rate Per 100,000'
               INNER JOIN NJ_crime AS CRM3 ON POV.county_name = CRM3.county_name AND CRM3.report_type = 'Number of Arrests'
               INNER JOIN (SELECT county_name, SUM(population) AS population FROM NJ_population GROUP BY county_name) AS POP ON POV.county_name = POP.county_name
               INNER JOIN (SELECT county_name, AVG(summativescore) AS school_rank FROM NJ_school_rating GROUP BY county_name) AS SCH ON POV.county_name = SCH.county_name'''
test = pd.read_sql_query(query, sqlite_connection)
test

In [ ]:
query = '''SELECT * from NJ_crime_detail'''
test = pd.read_sql_query(query, sqlite_connection)
test

In [ ]:
query = '''SELECT police_dept||"-"||report_type AS id ,report_type AS label, county_name||"-"||police_dept AS parent from NJ_crime_detail'''
test = pd.read_sql_query(query, sqlite_connection)
test

In [ ]:
query = '''SELECT county_name, 'murder' AS crime_type, murder AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'rape' AS crime_type, rape AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'robbery' AS crime_type, robbery AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'assault' AS crime_type, assault AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'burglary' AS crime_type, burglary AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'larceny' AS crime_type, larceny AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        UNION ALL SELECT county_name, 'auto_theft' AS crime_type, auto_theft AS count FROM NJ_crime WHERE report_type = 'Number of Offenses'
        '''
test = pd.read_sql_query(query, sqlite_connection)
test

In [ ]:
query1 = '''SELECT DISTINCT "NJ-"||county_name AS id , county_name AS label,  '' AS parent, AVG(summativescore) AS value FROM 
        NJ_school_rating GROUP BY 1,2,3'''
df1 = pd.read_sql(query1, sqlite_connection)
df1.head()

In [ ]:
query2 = '''SELECT DISTINCT county_name||"-"||district_name AS id ,district_name AS label,  "NJ-"||county_name AS parent, AVG(summativescore) AS value FROM 
        NJ_school_rating GROUP BY 1,2,3'''
df2 = pd.read_sql(query2, sqlite_connection)
df2.head()

In [ ]:
query4 = '''SELECT gradespan, AVG(summativescore) AS value FROM NJ_school_rating GROUP BY 1'''
df4 = pd.read_sql(query4, sqlite_connection)
df4

In [ ]:
query3 = '''SELECT DISTINCT district_name||"-"||school_name AS id ,school_name AS label,  county_name||"-"||district_name AS parent, AVG(summativescore) AS value FROM 
    NJ_school_rating GROUP BY 1,2,3'''
df3 = pd.read_sql(query3, sqlite_connection)
df3.head()

In [ ]:
query = '''SELECT county_name,district_name,gradespan,school_name,summativescore
            FROM 
              ( SELECT county_name,district_name,gradespan,school_name, summativescore,
                       ROW_NUMBER() OVER (PARTITION BY county_name
                                          ORDER BY summativescore DESC) AS rn
                FROM NJ_school_rating) AS tmp 
            WHERE rn <= 3
            ORDER BY county_name ; '''
test = pd.read_sql_query(query, sqlite_connection)
test

In [4]:
query = '''SELECT county_name,district_name,gradespan,school_name,summativescore FROM NJ_school_rating ORDER BY county_name,district_name,gradespan,school_name;'''
test = pd.read_sql_query(query, sqlite_connection)
test

2021-03-11 02:21:21,145 INFO sqlalchemy.engine.base.Engine SELECT county_name,district_name,gradespan,school_name,summativescore FROM NJ_school_rating ORDER BY county_name,district_name,gradespan,school_name;
2021-03-11 02:21:21,146 INFO sqlalchemy.engine.base.Engine ()


,county_name,district_name,gradespan,school_name,summativescore
0,ATLANTIC,ABSECON CITY,05-08,EMMA C ATTALES,43.95
1,ATLANTIC,ABSECON CITY,PK-04,H ASHTON MARSH,55.49
2,ATLANTIC,ATLANTIC CITY,09-12,ATLANTIC CITY HIGH SCHOOL,12.33
3,ATLANTIC,ATLANTIC CITY,KG-08,TEXAS AVENUE SCHOOL,31.79
4,ATLANTIC,ATLANTIC CITY,PK-05,BRIGHTON AVENUE SCHOOL,37.82
...,...,...,...,...,...
2007,WARREN,WARREN HILLS REGIONAL,07-08,WARREN HILLS REGIONAL MIDDLE SCHOOL,22.68
2008,WARREN,WARREN HILLS REGIONAL,09-12,WARREN HILLS REGIONAL HIGH SCHOOL,50.88
2009,WARREN,WASHINGTON BORO,03-06,WASHINGTON MEMORIAL ELEMENTARY SCHOOL,62.59
2010,WARREN,WASHINGTON TWP,PK-06,BRASS CASTLE SCHOOL,55.43


In [20]:
test = test.to_dict(orient='records')
pprint(test)

[{'county_name': 'ATLANTIC',
  'district_name': 'ABSECON CITY',
  'gradespan': '05-08',
  'school_name': 'EMMA C ATTALES',
  'summativescore': 43.95},
 {'county_name': 'ATLANTIC',
  'district_name': 'ABSECON CITY',
  'gradespan': 'PK-04',
  'school_name': 'H ASHTON MARSH',
  'summativescore': 55.49},
 {'county_name': 'ATLANTIC',
  'district_name': 'ATLANTIC CITY',
  'gradespan': '09-12',
  'school_name': 'ATLANTIC CITY HIGH SCHOOL',
  'summativescore': 12.33},
 {'county_name': 'ATLANTIC',
  'district_name': 'ATLANTIC CITY',
  'gradespan': 'KG-08',
  'school_name': 'TEXAS AVENUE SCHOOL',
  'summativescore': 31.79},
 {'county_name': 'ATLANTIC',
  'district_name': 'ATLANTIC CITY',
  'gradespan': 'PK-05',
  'school_name': 'BRIGHTON AVENUE SCHOOL',
  'summativescore': 37.82},
 {'county_name': 'ATLANTIC',
  'district_name': 'ATLANTIC CITY',
  'gradespan': 'PK-08',
  'school_name': 'CHELSEA HEIGHTS SCHOOL',
  'summativescore': 35.53},
 {'county_name': 'ATLANTIC',
  'district_name': 'ATLANTIC 

  'gradespan': 'KG-05',
  'school_name': 'RIDGE ELEMENTARY SCHOOL',
  'summativescore': 75.07},
 {'county_name': 'BERGEN',
  'district_name': 'RIDGEWOOD VILLAGE',
  'gradespan': 'KG-05',
  'school_name': 'SOMERVILLE ELEMENTARY SCHOOL',
  'summativescore': 71.57},
 {'county_name': 'BERGEN',
  'district_name': 'RIDGEWOOD VILLAGE',
  'gradespan': 'KG-05',
  'school_name': 'TRAVELL ELEMENTARY SCHOOL',
  'summativescore': 72.19},
 {'county_name': 'BERGEN',
  'district_name': 'RIDGEWOOD VILLAGE',
  'gradespan': 'KG-05',
  'school_name': 'WILLARD ELEMENTARY SCHOOL',
  'summativescore': 87.13},
 {'county_name': 'BERGEN',
  'district_name': 'RIVER DELL REGIONAL',
  'gradespan': '07-08',
  'school_name': 'RIVER DELL MIDDLE SCHOOL',
  'summativescore': 58.34},
 {'county_name': 'BERGEN',
  'district_name': 'RIVER DELL REGIONAL',
  'gradespan': '09-12',
  'school_name': 'RIVER DELL REGIONAL HIGH SCHOOL',
  'summativescore': 79.94},
 {'county_name': 'BERGEN',
  'district_name': 'RIVER EDGE BORO',
  

  'gradespan': 'KG-05',
  'school_name': 'KINGSTON ELEMENTARY SCHOOL',
  'summativescore': 56.6},
 {'county_name': 'CAMDEN',
  'district_name': 'CHERRY HILL TWP',
  'gradespan': 'KG-05',
  'school_name': 'RICHARD STOCKTON ELEMENTARY SCHOOL',
  'summativescore': 66.89},
 {'county_name': 'CAMDEN',
  'district_name': 'CHERRY HILL TWP',
  'gradespan': 'KG-05',
  'school_name': 'THOMAS PAINE ELEMENTARY SCHOOL',
  'summativescore': 43.65},
 {'county_name': 'CAMDEN',
  'district_name': 'CHERRY HILL TWP',
  'gradespan': 'KG-05',
  'school_name': 'WOODCREST ELEMENTARY SCHOOL',
  'summativescore': 40.29},
 {'county_name': 'CAMDEN',
  'district_name': 'CLEMENTON BORO',
  'gradespan': 'PK-08',
  'school_name': 'CLEMENTON ELEMENTARY SCHOOL',
  'summativescore': 52.64},
 {'county_name': 'CAMDEN',
  'district_name': 'COLLINGSWOOD BORO',
  'gradespan': '06-08',
  'school_name': 'COLLINGSWOOD MIDDLE SCHOOL',
  'summativescore': 32.25},
 {'county_name': 'CAMDEN',
  'district_name': 'COLLINGSWOOD BORO',


  'gradespan': '09-12',
  'school_name': 'NEWARK TECH',
  'summativescore': 81.62},
 {'county_name': 'ESSEX',
  'district_name': 'ESSEX CO VOC-TECH',
  'gradespan': '09-12',
  'school_name': 'NORTH 13TH ST TECH',
  'summativescore': 87.28},
 {'county_name': 'ESSEX',
  'district_name': 'ESSEX CO VOC-TECH',
  'gradespan': '09-12',
  'school_name': 'WEST CALDWELL TECH',
  'summativescore': 57.74},
 {'county_name': 'ESSEX',
  'district_name': 'ESSEX FELLS BORO',
  'gradespan': 'PK-06',
  'school_name': 'ESSEX FELLS ELEMENTARY SCHOOL',
  'summativescore': 93.91},
 {'county_name': 'ESSEX',
  'district_name': 'FAIRFIELD TWP',
  'gradespan': '04-06',
  'school_name': 'CHURCHILL',
  'summativescore': 61.57},
 {'county_name': 'ESSEX',
  'district_name': 'GLEN RIDGE BORO',
  'gradespan': '03-06',
  'school_name': 'RIDGEWOOD AVENUE SCHOOL',
  'summativescore': 84.7},
 {'county_name': 'ESSEX',
  'district_name': 'GLEN RIDGE BORO',
  'gradespan': '07-12',
  'school_name': 'GLEN RIDGE HIGH SCHOOL',
 

  'district_name': 'UNION CITY',
  'gradespan': '01-08',
  'school_name': 'SARA M. GILMORE ACADEMY',
  'summativescore': 80.68},
 {'county_name': 'HUDSON',
  'district_name': 'UNION CITY',
  'gradespan': '06-08',
  'school_name': 'EMERSON MIDDLE SCHOOL',
  'summativescore': 53.28},
 {'county_name': 'HUDSON',
  'district_name': 'UNION CITY',
  'gradespan': '07-08',
  'school_name': 'UNION HILL MIDDLE SCHOOL',
  'summativescore': 37.26},
 {'county_name': 'HUDSON',
  'district_name': 'UNION CITY',
  'gradespan': '09-12',
  'school_name': 'UNION CITY HIGH SCHOOL',
  'summativescore': 26.23},
 {'county_name': 'HUDSON',
  'district_name': 'UNION CITY',
  'gradespan': 'KG-05',
  'school_name': 'COLIN POWELL ELEMENTARY SCHOOL',
  'summativescore': 40.03},
 {'county_name': 'HUDSON',
  'district_name': 'UNION CITY',
  'gradespan': 'PK-04',
  'school_name': 'JEFFERSON ELEMENTARY SCHOOL',
  'summativescore': 71.18},
 {'county_name': 'HUDSON',
  'district_name': 'UNION CITY',
  'gradespan': 'PK-05'

 {'county_name': 'MONMOUTH',
  'district_name': 'ASBURY PARK CITY',
  'gradespan': 'PK-05',
  'school_name': 'THURGOOD MARSHALL ELEMENTARY SCHOOL',
  'summativescore': 14.24},
 {'county_name': 'MONMOUTH',
  'district_name': 'ATLANTIC HIGHLANDS BORO',
  'gradespan': 'PK-06',
  'school_name': 'ATLANTIC HIGHLANDS ELEMENTARY SCHOOL',
  'summativescore': 79.03},
 {'county_name': 'MONMOUTH',
  'district_name': 'AVON BORO',
  'gradespan': 'KG-08',
  'school_name': 'AVON ELEMENTARY SCHOOL',
  'summativescore': 64.06},
 {'county_name': 'MONMOUTH',
  'district_name': 'BELMAR BORO',
  'gradespan': 'PK-08',
  'school_name': 'BELMAR ELEMENTARY',
  'summativescore': 43.65},
 {'county_name': 'MONMOUTH',
  'district_name': 'BRADLEY BEACH BORO',
  'gradespan': 'PK-08',
  'school_name': 'BRADLEY BEACH ELEMENTARY SCHOOL',
  'summativescore': 38.74},
 {'county_name': 'MONMOUTH',
  'district_name': 'BRIELLE BORO',
  'gradespan': 'PK-08',
  'school_name': 'BRIELLE ELEMENTARY SCHOOL',
  'summativescore': 86.

  'gradespan': '06-08',
  'school_name': 'LONG VALLEY MIDDLE SCHOOL',
  'summativescore': 56.59},
 {'county_name': 'MORRIS',
  'district_name': 'WASHINGTON TWP',
  'gradespan': 'KG-05',
  'school_name': 'OLD FARMERS ROAD SCHOOL',
  'summativescore': 88.59},
 {'county_name': 'MORRIS',
  'district_name': 'WASHINGTON TWP',
  'gradespan': 'PK-05',
  'school_name': 'BENEDICT A. CUCINELLA SCHOOL',
  'summativescore': 62.76},
 {'county_name': 'MORRIS',
  'district_name': 'WASHINGTON TWP',
  'gradespan': 'PK-05',
  'school_name': 'FLOCKTOWN-KOSSMANN ELEMENTARY SCHOOL',
  'summativescore': 59.08},
 {'county_name': 'MORRIS',
  'district_name': 'WEST MORRIS REGIONAL',
  'gradespan': '09-12',
  'school_name': 'WEST MORRIS CENTRAL HIGH SCHOOL',
  'summativescore': 88.49},
 {'county_name': 'MORRIS',
  'district_name': 'WEST MORRIS REGIONAL',
  'gradespan': '09-12',
  'school_name': 'WEST MORRIS MENDHAM HIGH SCHOOL',
  'summativescore': 79.3},
 {'county_name': 'MORRIS',
  'district_name': 'WHARTON BO

 {'county_name': 'SALEM',
  'district_name': 'PENNSVILLE',
  'gradespan': '04-05',
  'school_name': 'PENN BEACH ELEMENTARY SCHOOL',
  'summativescore': 70.41},
 {'county_name': 'SALEM',
  'district_name': 'PENNSVILLE',
  'gradespan': '06-08',
  'school_name': 'PENNSVILLE MIDDLE SCHOOL',
  'summativescore': 26.96},
 {'county_name': 'SALEM',
  'district_name': 'PENNSVILLE',
  'gradespan': '09-12',
  'school_name': 'PENNSVILLE MEMORIAL HIGH SCHOOL',
  'summativescore': 23.34},
 {'county_name': 'SALEM',
  'district_name': 'PITTSGROVE TWP',
  'gradespan': '06-08',
  'school_name': 'PITTSGROVE TOWNSHIP MIDDLE SCHOOL',
  'summativescore': 68.44},
 {'county_name': 'SALEM',
  'district_name': 'PITTSGROVE TWP',
  'gradespan': '09-12',
  'school_name': 'ARTHUR P. SCHALICK HIGH SCHOOL',
  'summativescore': 49.41},
 {'county_name': 'SALEM',
  'district_name': 'PITTSGROVE TWP',
  'gradespan': 'KG-05',
  'school_name': 'OLIVET ELEMENTARY SCHOOL',
  'summativescore': 50.55},
 {'county_name': 'SALEM',


  'district_name': 'SCOTCH PLAINS-FANWOOD REG',
  'gradespan': 'PK-04',
  'school_name': 'EVERGREEN ELEMENTARY SCHOOL',
  'summativescore': 83.98},
 {'county_name': 'UNION',
  'district_name': 'SCOTCH PLAINS-FANWOOD REG',
  'gradespan': 'PK-04',
  'school_name': 'HOWARD B. BRUNNER ELEMENTARY SCHOOL',
  'summativescore': 78.03},
 {'county_name': 'UNION',
  'district_name': 'SCOTCH PLAINS-FANWOOD REG',
  'gradespan': 'PK-04',
  'school_name': 'J. ACKERMAN COLES ELEMENTARY SCHOOL',
  'summativescore': 84.12},
 {'county_name': 'UNION',
  'district_name': 'SCOTCH PLAINS-FANWOOD REG',
  'gradespan': 'PK-04',
  'school_name': 'SCHOOL ONE ELEMENTARY',
  'summativescore': 80.48},
 {'county_name': 'UNION',
  'district_name': 'SPRINGFIELD TWP',
  'gradespan': '03-05',
  'school_name': 'JAMES CALDWELL ELEMENTARY SCHOOL',
  'summativescore': 60.59},
 {'county_name': 'UNION',
  'district_name': 'SPRINGFIELD TWP',
  'gradespan': '03-05',
  'school_name': 'THELMA L. SANDMEIER ELEMENTARY SCHOOL',
  'su

In [24]:
with open("data/school2.json", "w",encoding ='utf8') as outfile:  
    json.dump(test, outfile, indent = 1) 

In [15]:
test.dtypes

county_name        object
district_name      object
gradespan          object
school_name        object
summativescore    float64
dtype: object

In [5]:
flare = {}
flare["name"] = "flare"
flare["description"] = "flare"
flare

{'name': 'flare', 'description': 'flare'}

In [6]:
counties = list(test['county_name'].unique())
# counties

In [17]:
children = []
for i in range(len(counties)):
    child1 = {}
    child1["name"] = counties[i]
    child1["description"] = counties[i]
    district = list(test['district_name'].loc[test['county_name']==counties[i]].unique())
    child2_list = []
    for k in range(len(district)):
        child2 = {}
        child2["name"] = district[k]
        child2["description"] = district[k]
        child3_list = []
        gradespan = list(test['gradespan'].loc[(test['county_name']==counties[i]) & (test['district_name'] == district[k])].unique())
        for j in range(len(gradespan)):
            child3 = {}
            child3["name"] = gradespan[j]
            child3["description"] = gradespan[j]
            child4_list = []
            for index,row in test.loc[(test['county_name']==counties[i]) & (test['district_name'] == district[k]) & (test['gradespan'] == gradespan[j])].iterrows():
                child4 = {}
                child4["name"] = row["school_name"]
                child4["description"] = row["school_name"]
                child4["size"] = round(row["summativescore"])
                child4_list.append(child4)
            child3["children"] = child4_list
            child3_list.append(child3)
        child2["children"] = child3_list
        child2_list.append(child2)
    child1["children"] = child2_list
    children.append(child1)
children

[{'name': 'ATLANTIC',
  'description': 'ATLANTIC',
  'children': [{'name': 'ABSECON CITY',
    'description': 'ABSECON CITY',
    'children': [{'name': '05-08',
      'description': '05-08',
      'children': [{'name': 'EMMA C ATTALES',
        'description': 'EMMA C ATTALES',
        'size': 44}]},
     {'name': 'PK-04',
      'description': 'PK-04',
      'children': [{'name': 'H ASHTON MARSH',
        'description': 'H ASHTON MARSH',
        'size': 55}]}]},
   {'name': 'ATLANTIC CITY',
    'description': 'ATLANTIC CITY',
    'children': [{'name': '09-12',
      'description': '09-12',
      'children': [{'name': 'ATLANTIC CITY HIGH SCHOOL',
        'description': 'ATLANTIC CITY HIGH SCHOOL',
        'size': 12}]},
     {'name': 'KG-08',
      'description': 'KG-08',
      'children': [{'name': 'TEXAS AVENUE SCHOOL',
        'description': 'TEXAS AVENUE SCHOOL',
        'size': 32}]},
     {'name': 'PK-05',
      'description': 'PK-05',
      'children': [{'name': 'BRIGHTON AVENUE S

In [18]:
flare["children"] = children
flare

{'name': 'flare',
 'description': 'flare',
 'children': [{'name': 'ATLANTIC',
   'description': 'ATLANTIC',
   'children': [{'name': 'ABSECON CITY',
     'description': 'ABSECON CITY',
     'children': [{'name': '05-08',
       'description': '05-08',
       'children': [{'name': 'EMMA C ATTALES',
         'description': 'EMMA C ATTALES',
         'size': 44}]},
      {'name': 'PK-04',
       'description': 'PK-04',
       'children': [{'name': 'H ASHTON MARSH',
         'description': 'H ASHTON MARSH',
         'size': 55}]}]},
    {'name': 'ATLANTIC CITY',
     'description': 'ATLANTIC CITY',
     'children': [{'name': '09-12',
       'description': '09-12',
       'children': [{'name': 'ATLANTIC CITY HIGH SCHOOL',
         'description': 'ATLANTIC CITY HIGH SCHOOL',
         'size': 12}]},
      {'name': 'KG-08',
       'description': 'KG-08',
       'children': [{'name': 'TEXAS AVENUE SCHOOL',
         'description': 'TEXAS AVENUE SCHOOL',
         'size': 32}]},
      {'name': 'PK

In [19]:
with open("data/school.json", "w",encoding ='utf8') as outfile:  
    json.dump(flare, outfile, indent = 1) 

In [25]:
sqlite_connection.close()

# JSON creation

In [ ]:
crime_dict = crime_df.to_dict(orient='records')
pprint(crime_dict)

In [ ]:
metadata_dict = metadata_df.to_dict(orient='records')
pprint(metadata_dict)

In [ ]:
import json

In [ ]:
school_dict = school_df.to_dict(orient='records')
pprint(school_dict)

In [ ]:
data_json= {}
data_json["metadata"] = metadata_dict
data_json["school"] = school_dict
data_json["crime"] = crime_dict

# Linear regression

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
import scipy.stats as st
from scipy.stats import linregress
from scipy.stats import sem

In [ ]:
test.columns

In [ ]:
test.dtypes

In [ ]:
# Calculate the correlation coefficient and linear regression model 
x = test['median_hh_income'].astype(float)
# x = test['school_rank']
y = test['total_offense']

correlation = st.pearsonr(x,y)

# Print the answer to above calculation
print(f"""The correlation between school_rank and total_offense is {round(correlation[0],2)}.""")

# Calculate linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot linear regression on top of scatter plot
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")

# Annotate linear regression
plt.annotate(line_eq,(20,36),fontsize=16,color="red")

# Add labels and title to plot
plt.xlabel("median_hh_income")
plt.ylabel("total_offense")
plt.title('total_offense by median_hh_income')
# plt.savefig("Images/scatter_line_plot.png")
plt.show()